In [92]:
import csv
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions 

# initiate selenium webdriver and get the page
driver = webdriver.Chrome()
driver.get('https://www.amazon.com/Tineco-Cordless-Lightweight-Powerful-Multi-Surface/product-reviews/B082VRFWB8/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&language=en_US&reviewerType=all_reviews')
driver.implicitly_wait(5)

# list used to store data
rows = []

# initiate beautiful soup parser
html = driver.page_source
soup = BeautifulSoup(html.encode('utf-8'),"html.parser")


# find review card
divs = soup.select("body div#a-page div#cm_cr-review_list div[data-hook=review]")

# iterate review card to get data
for idx, div in enumerate(divs):
    # initiate variables
    account_name = None
    score = None
    title = None
    Location = None
    date = None
    product = None
    purchase_status = None
    review_content = None
    number_like = None

    # get account name
    span_account = div.select("div.a-profile-content span.a-profile-name")
    if span_account != []:
        account_name = span_account[0].get_text()

    # get score
    a_score = div.select("div.a-row i[data-hook=review-star-rating] span.a-icon-alt")
    if a_score != []:
        score = a_score[0].get_text()

    # get title
    span_title = div.select("a[data-hook=review-title] span")
    if span_title != []:
        title = span_title[0].get_text()

    # get location and date
    span_location_date = div.select("span[data-hook=review-date]")
    if span_location_date != []:
        span_content = span_location_date[0].get_text()
    Location = re.findall(r'Reviewed in ([^o]+)', span_content)[0].strip()
    date = re.findall(r'on ([^"]+)', span_content)[0].strip()

    # get product
    a_product = div.select("div a[data-hook=format-strip]")
    if a_product != []:
        product = a_product[0].get_text()

    # get purchase status
    span_purchase = div.select("span.a-declarative span[data-hook=avp-badge]")
    if span_purchase != []:
        purchase_status = span_purchase[0].get_text()

    # get review content
    span_content = div.select("span[data-hook=review-body] span")
    if span_content != []:
        review_content = span_content[0].get_text().strip()

    # get number of likes
    span_like = div.select("span.cr-vote span[data-hook=helpful-vote-statement]")
    if span_like != []:
        number_like = span_like[0].get_text()[:-26]

    rows.append([account_name, score, title, Location, date, product, purchase_status, review_content, number_like])

#     try:
#         next_link = soup.select("span.a-declarative ul.a-pagination li.a-last a")[0]['href']
#     except:
#         break

headers = ['Account_name','Score','Title','Location','Date', 'Product', 'Purchase_Status', 'Review_content', 'Like']

with open('test.csv','w', encoding="utf-8", newline='')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)
